In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import dask.array as da
import pyproj
import sys
from joblib import Parallel, delayed
from tqdm import tqdm
from pycontrails import Flight, Fleet, MetDataset
from pycontrails.core import models
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.physics import geo, thermo, units, constants
from pycontrails.models.gpat.gpat import GPAT

In [22]:
job_id = 809354

In [23]:
params = pd.read_pickle("outputs/" + repr(job_id) + "/params_" + repr(job_id) + ".pkl")
params

{'fl_params': {'t0_fl': Timestamp('2022-01-20 13:00:00'),
  'rt_fl': Timedelta('0 days 01:00:00'),
  'ts_fl': Timedelta('0 days 00:02:00'),
  'ac_type': 'A320',
  'fl0_speed': 100.0,
  'fl0_heading': 0.0,
  'fl0_coords0': (0.1, 0.125, 12500),
  'sep_dist': (5000, 2000, 0),
  'n_ac': 1},
 'plume_params': {'dt_integration': Timedelta('0 days 00:02:00'),
  'max_age': Timedelta('0 days 01:00:00'),
  'depth': 50.0,
  'width': 50.0,
  'shear': 0.01,
  'hres_pl': 0.05,
  'vres_pl': 500},
 'sim_params': {'t0_sim': Timestamp('2022-01-20 12:00:00'),
  'rt_sim': Timedelta('0 days 03:00:00'),
  'ts_sim': Timedelta('0 days 00:00:20'),
  'lat_bounds': (0.0, 1.0),
  'lon_bounds': (0.0, 1.0),
  'alt_bounds': (12000, 13000),
  'hres_sim': 0.05,
  'vres_sim': 500,
  'eastward_wind': 0.0,
  'northward_wind': 0.0,
  'lagrangian_tendency_of_air_pressure': 0.0,
  'species_out': array([3, 4, 5, 6, 8, 9])}}

In [24]:
fl = pd.read_pickle("outputs/" + repr(job_id) + "/fl_" + repr(job_id) + ".pkl")
fl

,longitude,latitude,altitude,time,air_temperature,specific_humidity,true_airspeed,flight_id,aircraft_mass,engine_efficiency,...,NO,NO2,CO,HCHO,CH3CHO,C2H4,C3H6,C2H2,BENZENE,waypoint
0,0.125,0.100000,12500.0,2022-01-20 13:00:00,212.263516,0.000219,100.564816,0.0,62090.764980,0.100054,...,2.048473,0.107814,0.062195,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,0
1,0.125,0.208524,12500.0,2022-01-20 13:02:00,212.260586,0.000220,100.564816,0.0,61949.103196,0.100054,...,2.048571,0.107820,0.062195,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,1
2,0.125,0.317048,12500.0,2022-01-20 13:04:00,212.257656,0.000221,100.564816,0.0,61807.434026,0.100055,...,2.048670,0.107825,0.062196,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,2
3,0.125,0.425573,12500.0,2022-01-20 13:06:00,212.254726,0.000223,100.564816,0.0,61665.757471,0.100055,...,2.048769,0.107830,0.062196,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,3
4,0.125,0.534097,12500.0,2022-01-20 13:08:00,212.251796,0.000224,100.564816,0.0,61524.073531,0.100055,...,2.048867,0.107835,0.062196,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,4
5,0.125,0.642621,12500.0,2022-01-20 13:10:00,212.248865,0.000226,100.564816,0.0,61382.382203,0.100056,...,2.048966,0.107840,0.062197,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,5
6,0.125,0.751145,12500.0,2022-01-20 13:12:00,212.245935,0.000227,100.564816,0.0,61240.683488,0.100056,...,2.049065,0.107846,0.062197,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,6
7,0.125,0.859670,12500.0,2022-01-20 13:14:00,212.243005,0.000228,100.564816,0.0,61098.977386,0.100056,...,2.049065,0.107846,0.062197,0.000728,0.000243,0.00091,0.000243,0.000243,0.000121,7


In [25]:
pl = pd.read_pickle("outputs/" + repr(job_id) + "/pl_" + repr(job_id) + ".pkl")
pl

,flight_id,waypoint,fuel_flow,true_airspeed,CO2,H2O,SO2,NO,NO2,CO,...,longitude,latitude,level,width,heading,sigma_yy,sigma_zz,sin_a,cos_a,altitude
0,0.0,0,1.180515,100.564816,447.651238,174.243994,0.118996,2.048473,0.107814,0.062195,...,0.125,0.100000,178.645172,83.609285,NaN,8.738141e+02,544.404306,NaN,NaN,12500.0
1,0.0,0,1.180515,100.564816,447.651238,174.243994,0.118996,2.048473,0.107814,0.062195,...,0.125,0.100000,178.645172,154.997098,90.0,3.003013e+03,776.308612,1.0,6.123234e-17,12500.0
30,0.0,1,1.180576,100.564816,447.674575,174.253078,0.119002,2.048571,0.107820,0.062195,...,0.125,0.208524,178.645172,83.608924,90.0,8.738065e+02,544.388565,1.0,6.123234e-17,12500.0
2,0.0,0,1.180515,100.564816,447.651238,174.243994,0.118996,2.048473,0.107814,0.062195,...,0.125,0.100000,178.645172,242.783522,90.0,7.367980e+03,1008.212918,1.0,6.123234e-17,12500.0
31,0.0,1,1.180576,100.564816,447.674575,174.253078,0.119002,2.048571,0.107820,0.062195,...,0.125,0.208524,178.645172,154.995538,90.0,3.002952e+03,776.277130,1.0,6.123234e-17,12500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0.0,6,1.180884,100.564816,447.791283,174.298506,0.119033,2.049065,0.107846,0.062197,...,0.125,0.751145,178.645172,4973.928877,90.0,3.092496e+06,7034.987019,1.0,6.123234e-17,12500.0
237,0.0,7,1.180884,100.564816,447.791283,174.298506,0.119033,2.049065,0.107846,0.062197,...,0.125,0.859670,178.645172,4729.074671,90.0,2.795518e+06,6802.736781,1.0,6.123234e-17,12500.0
209,0.0,6,1.180884,100.564816,447.791283,174.298506,0.119033,2.049065,0.107846,0.062197,...,0.125,0.751145,178.645172,5222.695678,90.0,3.409569e+06,7266.796916,1.0,6.123234e-17,12500.0
238,0.0,7,1.180884,100.564816,447.791283,174.298506,0.119033,2.049065,0.107846,0.062197,...,0.125,0.859670,178.645172,4973.780819,90.0,3.092312e+06,7034.530952,1.0,6.123234e-17,12500.0


In [26]:
chem = xr.open_dataset("outputs/" + repr(job_id) +"/chem_" + repr(job_id) + ".nc")
chem["emi"].sel(emi_species="NO", time=fl["time"][0]).sel(level=178.6, method="nearest").values

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.,

In [27]:

"""Check if mass is conserved in the box model."""

mm = [30.01, 46.01, 28.01, 30.03, 44.05, 28.05, 42.08, 26.04, 78.11]  # g/mol
NA = 6.022e23  # Avogadro's number

total_vector_mass = 0
total_grid_mass = 0
for ts, time in enumerate(fl["time"][:-1]):

    # grab vector data
    for s, emi_species in enumerate(["NO"]):# self.boxm_ds_unstacked["emi_species"].data):
        
        vector_mass = fl[emi_species][ts]
    
        total_vector_mass += vector_mass

        # grab plume mass from grid data
        grid_concs = chem["emi"].sel(emi_species=emi_species, time=time).sel(level=178.6, method="nearest")

        print(grid_concs)

        grid_concs_over_zero = grid_concs.where(grid_concs > 0, drop=True)

        grid_mass = grid_concs_over_zero \
            * chem["M"].sel(time=time).sel(level=178.6, method="nearest") \
            * 1e-9 \
            * (mm[s] / NA) \
            * params["chem_params"]["vres_chem"] \
            * units.latitude_distance_to_m(params["chem_params"]["hres_chem"]) \
            * units.longitude_distance_to_m(params["chem_params"]["hres_chem"], (params["chem_params"]["lat_bounds"][0] + params["chem_params"]["lat_bounds"][1]) / 2) \
            * 1E+03 # convert to kg/m^3
            

        grid_mass_sum = grid_mass.sum().values

        total_grid_mass += grid_mass_sum
    
    print(total_vector_mass, total_grid_mass)

<xarray.DataArray 'emi' (longitude: 21, latitude: 21)> Size: 4kB
[441 values with dtype=float64]
Coordinates:
    level         float64 8B 178.6
  * longitude     (longitude) float64 168B 0.0 0.05 0.1 0.15 ... 0.9 0.95 1.0
  * latitude      (latitude) float64 168B 0.0 0.05 0.1 0.15 ... 0.9 0.95 1.0
    time          datetime64[ns] 8B 2022-01-20T13:00:00
    air_pressure  (longitude, latitude) float64 4kB ...
    altitude      (longitude, latitude) float64 4kB ...
    emi_species   <U7 28B 'NO'


ValueError: Cannot apply_along_axis when any iteration dimensions are 0

In [ ]:
from shapely import Polygon

poly1 = Polygon([])